# Flight Data Cleaning




## Reading in data

In [24]:
### Mounting Drive
from google.colab import drive
drive.mount('/content/gdrive')
### Useful packages for cleaning / EDA
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder # one-hot encodes string features
from sklearn.preprocessing import StandardScaler # Scaling regressive features
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report # creates nice report of model performance
from sklearn.model_selection import GridSearchCV # tests different parameters
import seaborn as sns # nice library for visualizations
from datetime import datetime

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [25]:
### Getting file paths
files = !ls 'gdrive/My Drive/correct_data'
csv_files = files[0].split(sep=' ')
csv_files = ['gdrive/My Drive/correct_data/' + f for f in csv_files if f != '']
csv_files

['gdrive/My Drive/correct_data/ls:',
 'gdrive/My Drive/correct_data/cannot',
 'gdrive/My Drive/correct_data/access',
 "gdrive/My Drive/correct_data/'gdrive/My",
 "gdrive/My Drive/correct_data/Drive/correct_data':",
 'gdrive/My Drive/correct_data/No',
 'gdrive/My Drive/correct_data/such',
 'gdrive/My Drive/correct_data/file',
 'gdrive/My Drive/correct_data/or',
 'gdrive/My Drive/correct_data/directory']

In [26]:

### Read into pandas df's, skip first few rows (metadata)
AA = pd.read_csv('gdrive/My Drive/raw_correct_data/AA.csv',skiprows=7)
AS = pd.read_csv('gdrive/My Drive/raw_correct_data/AS.csv',skiprows=7)
DL = pd.read_csv('gdrive/My Drive/raw_correct_data/DL.csv',skiprows=7)
WN = pd.read_csv('gdrive/My Drive/raw_correct_data/WN.csv',skiprows=7)
UA = pd.read_csv('gdrive/My Drive/raw_correct_data/UA.csv',skiprows=7)
JB = pd.read_csv('gdrive/My Drive/raw_correct_data/JB.csv',skiprows=7)
SA = pd.read_csv('gdrive/My Drive/raw_correct_data/SA.csv',skiprows=7)
airlines = [AA, AS, DL, WN, UA, JB, SA]

## Exploratory Data Analysis

In [27]:
print(AS.shape)
AS.head(10)

(36878, 17)


,Carrier Code,Date (MM/DD/YYYY),Flight Number,Tail Number,Destination Airport,Scheduled departure time,Actual departure time,Scheduled elapsed time (Minutes),Actual elapsed time (Minutes),Departure delay (Minutes),Wheels-off time,Taxi-Out time (Minutes),Delay Carrier (Minutes),Delay Weather (Minutes),Delay National Aviation System (Minutes),Delay Security (Minutes),Delay Late Aircraft Arrival (Minutes)
0,AS,01/01/2021,6.0,N513AS,DCA,13:10,12:57,288.0,284.0,-13.0,13:15,18.0,0.0,0.0,0.0,0.0,0.0
1,AS,01/01/2021,193.0,N495AS,ANC,19:55,19:47,348.0,355.0,-8.0,20:06,19.0,0.0,0.0,0.0,0.0,0.0
2,AS,01/01/2021,312.0,N461AS,TPA,14:30,14:15,290.0,280.0,-15.0,14:35,20.0,0.0,0.0,0.0,0.0,0.0
3,AS,01/01/2021,369.0,N361VA,PDX,17:25,17:12,150.0,141.0,-13.0,17:32,20.0,0.0,0.0,0.0,0.0,0.0
4,AS,01/01/2021,378.0,N579AS,RSW,14:10,13:47,285.0,284.0,-23.0,14:06,19.0,0.0,0.0,0.0,0.0,0.0
5,AS,01/01/2021,503.0,N848VA,SEA,07:15,07:06,180.0,164.0,-9.0,07:20,14.0,0.0,0.0,0.0,0.0,0.0
6,AS,01/01/2021,512.0,N468AS,EWR,07:45,07:38,320.0,312.0,-7.0,07:50,12.0,0.0,0.0,0.0,0.0,0.0
7,AS,01/01/2021,525.0,N923VA,SEA,09:15,09:09,180.0,154.0,-6.0,09:23,14.0,0.0,0.0,0.0,0.0,0.0
8,AS,01/01/2021,705.0,N533AS,SEA,19:35,19:30,175.0,171.0,-5.0,20:02,32.0,0.0,0.0,0.0,0.0,0.0
9,AS,01/01/2021,813.0,N518AS,OGG,17:50,17:39,365.0,356.0,-11.0,17:55,16.0,0.0,0.0,0.0,0.0,0.0


In [28]:
AS.describe()

,Flight Number,Scheduled elapsed time (Minutes),Actual elapsed time (Minutes),Departure delay (Minutes),Taxi-Out time (Minutes),Delay Carrier (Minutes),Delay Weather (Minutes),Delay National Aviation System (Minutes),Delay Security (Minutes),Delay Late Aircraft Arrival (Minutes)
count,36877.000000,36877.000000,36877.000000,36877.000000,36877.000000,36877.000000,36877.000000,36877.000000,36877.000000,36877.000000
mean,779.818803,208.520189,200.335060,5.662934,21.607940,2.951894,0.110719,3.140358,0.033734,3.195949
std,441.803859,82.359332,86.419049,32.638934,7.897464,19.130169,3.518931,13.444241,1.262877,18.810214
min,6.000000,69.000000,0.000000,-27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,344.000000,154.000000,149.000000,-8.000000,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,817.000000,175.000000,168.000000,-4.000000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1178.000000,296.000000,289.000000,4.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1499.000000,395.000000,520.000000,1007.000000,114.000000,983.000000,259.000000,435.000000,139.000000,661.000000


In [29]:
AS.isnull().sum()

,0
Carrier Code,0
Date (MM/DD/YYYY),1
Flight Number,1
Tail Number,1
Destination Airport,1
Scheduled departure time,1
Actual departure time,1
Scheduled elapsed time (Minutes),1
Actual elapsed time (Minutes),1
Departure delay (Minutes),1


In [30]:
### Time in df are strings (we want pandas date times for Date/ scheduled departure time, etc. )
# print(AS.applymap(type))

## Cleaning the data


## Error in time date below:
* 1 row in the Wheels-off time contains a time that is not within the valid 0:00 - 23:59 date time format (>24:00). This likely represents the next day.
* We could just convert it by subtracting 24:00 to get the actual valid time.
* Ex. 24:10 - 24:00 = 00:10 (12:10 AM next day).


### AA

In [31]:
AA.rename(
    columns={
        "Carrier Code (IATA)": "carrier_code",
        "Date (MM/DD/YYYY)": "flight_date",
        "Flight Number": "flight_number",
        "Tail Number": "tail_number",
        "Destination Airport": "destination_airport",
        "Scheduled departure time": "scheduled_dep_time",
        "Actual departure time": "actual_dep_time",
        "Scheduled elapsed time (Minutes)": "scheduled_elapsed",
        "Actual elapsed time (Minutes)": "actual_elapsed",
        "Departure delay (Minutes)": "dep_delay",
        "Wheels-off time": "dep_delay_wheels_off",  # Updated key to match CSV header
        "Taxi-Out time (Minutes)": "taxi_out_time",
        "Delay Carrier (Minutes)": "delay_carrier",
        "Delay Weather (Minutes)": "delay_weather",
        "Delay National Aviation System (Minutes)": "delay_nas",
        "Delay Security (Minutes)": "delay_security",
        "Delay Late Aircraft Arrival (Minutes)": "delay_late_aircraft"
    },
    inplace=True
)

AA['flight_date'] = pd.to_datetime(AA['flight_date'], format='%m/%d/%Y', errors='coerce')
AA['flight_number'] = pd.to_numeric(AA['flight_number'], errors='coerce')



AA["scheduled_dep_time"] = AA["scheduled_dep_time"].astype(str).str.strip()
AA["scheduled_dep_time"] = (
    pd.to_datetime(AA["scheduled_dep_time"], format="%H:%M", errors="coerce")
    .dt.time
)


###################

AA['scheduled_dep_hour'] = AA['scheduled_dep_time'].apply(
    lambda t: t.hour if pd.notnull(t) else None
)

# (Optional) Convert the new column to numeric (int) if desired
AA['scheduled_dep_hour'] = pd.to_numeric(AA['scheduled_dep_hour'], errors='coerce')

# (Optional) If you want a true integer column that can handle missing values,
# convert to a nullable integer type:
AA['scheduled_dep_hour'] = AA['scheduled_dep_hour'].astype('Int64')



##################

AA["actual_dep_time"] = (
    pd.to_datetime(
        AA["actual_dep_time"],
        format="%H:%M",
        errors="coerce"
    )
    .dt.time
)

# Convert to numeric in case there are any stray strings
AA["dep_delay_wheels_off"] = pd.to_datetime(
    AA["dep_delay_wheels_off"], format="%H:%M", errors="coerce"
).dt.time

AA['dep_delay_wheels_off_hour'] = AA['dep_delay_wheels_off'].apply(lambda x: x.hour if pd.notnull(x) else None)

# Convert the new 'dep_delay_wheels_off_hour' column to numeric type
AA['dep_delay_wheels_off_hour'] = pd.to_numeric(AA['dep_delay_wheels_off_hour'], errors='coerce')


numeric_cols = [
    'scheduled_elapsed', 'actual_elapsed',
    'dep_delay', 'taxi_out_time', 'delay_carrier',
    'delay_weather', 'delay_nas', 'delay_security', 'delay_late_aircraft'
]

for col in numeric_cols:
    AA[col] = pd.to_numeric(AA[col], errors='coerce')

# Handle missing data
AA.dropna(subset=['flight_date', 'flight_number', 'destination_airport'], inplace=True)

# Remove duplicates
AA.drop_duplicates(inplace=True)

# Final check
AA.info()
AA.head()

<class 'pandas.core.frame.DataFrame'>
Index: 117697 entries, 0 to 117696
Data columns (total 19 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Carrier Code               117697 non-null  object        
 1   flight_date                117697 non-null  datetime64[ns]
 2   flight_number              117697 non-null  float64       
 3   tail_number                117688 non-null  object        
 4   destination_airport        117697 non-null  object        
 5   scheduled_dep_time         117697 non-null  object        
 6   actual_dep_time            117563 non-null  object        
 7   scheduled_elapsed          117697 non-null  float64       
 8   actual_elapsed             117697 non-null  float64       
 9   dep_delay                  117697 non-null  float64       
 10  dep_delay_wheels_off       117590 non-null  object        
 11  taxi_out_time              117697 non-null  float64      

,Carrier Code,flight_date,flight_number,tail_number,destination_airport,scheduled_dep_time,actual_dep_time,scheduled_elapsed,actual_elapsed,dep_delay,dep_delay_wheels_off,taxi_out_time,delay_carrier,delay_weather,delay_nas,delay_security,delay_late_aircraft,scheduled_dep_hour,dep_delay_wheels_off_hour
0,AA,2021-01-01,2.0,N774AN,JFK,08:30:00,08:23:00,311.0,313.0,-7.0,08:39:00,16.0,0.0,0.0,0.0,0.0,0.0,8,8.0
1,AA,2021-01-01,4.0,N772AN,JFK,21:15:00,21:11:00,304.0,308.0,-4.0,21:33:00,22.0,0.0,0.0,0.0,0.0,0.0,21,21.0
2,AA,2021-01-01,31.0,N405AN,HNL,08:45:00,08:42:00,364.0,327.0,-3.0,08:51:00,9.0,0.0,0.0,0.0,0.0,0.0,8,8.0
3,AA,2021-01-01,52.0,N188US,CLT,06:03:00,05:58:00,276.0,276.0,-5.0,06:11:00,13.0,0.0,0.0,0.0,0.0,0.0,6,6.0
4,AA,2021-01-01,59.0,N410AN,KOA,09:00:00,08:53:00,355.0,314.0,-7.0,09:02:00,9.0,0.0,0.0,0.0,0.0,0.0,9,9.0


In [32]:
AA['month'] = AA['flight_date'].dt.month

# Extract the day of the month (1–31)
AA['day'] = AA['flight_date'].dt.day

AA.to_csv('gdrive/My Drive/clean_data_v2/AA.csv', index=False)

### AS

In [33]:
# Rename columns using exact names from the CSV
AS.rename(
    columns={
        "Carrier Code (IATA)": "carrier_code",
        "Date (MM/DD/YYYY)": "flight_date",
        "Flight Number": "flight_number",
        "Tail Number": "tail_number",
        "Destination Airport": "destination_airport",
        "Scheduled departure time": "scheduled_dep_time",
        "Actual departure time": "actual_dep_time",
        "Scheduled elapsed time (Minutes)": "scheduled_elapsed",
        "Actual elapsed time (Minutes)": "actual_elapsed",
        "Departure delay (Minutes)": "dep_delay",
        "Wheels-off time": "dep_delay_wheels_off",  # Updated key to match CSV header
        "Taxi-Out time (Minutes)": "taxi_out_time",
        "Delay Carrier (Minutes)": "delay_carrier",
        "Delay Weather (Minutes)": "delay_weather",
        "Delay National Aviation System (Minutes)": "delay_nas",
        "Delay Security (Minutes)": "delay_security",
        "Delay Late Aircraft Arrival (Minutes)": "delay_late_aircraft"
    },
    inplace=True
)


# Convert data types
AS['flight_date'] = pd.to_datetime(AS['flight_date'], format='%m/%d/%Y', errors='coerce')
AS['flight_number'] = pd.to_numeric(AS['flight_number'], errors='coerce')

# Convert HHMM columns to time
AS["scheduled_dep_time"] = (
    pd.to_datetime(
        AS["scheduled_dep_time"],    # already something like "13:10"
        format="%H:%M",
        errors="coerce"
    )
    .dt.time  # convert to a Python time object
)


###################

AS['scheduled_dep_hour'] = AS['scheduled_dep_time'].apply(
    lambda t: t.hour if pd.notnull(t) else None
)

# (Optional) Convert the new column to numeric (int) if desired
AS['scheduled_dep_hour'] = pd.to_numeric(AS['scheduled_dep_hour'], errors='coerce')

# (Optional) If you want a true integer column that can handle missing values,
# convert to a nullable integer type:
AS['scheduled_dep_hour'] = AS['scheduled_dep_hour'].astype('Int64')



##################

AS["actual_dep_time"] = (
    pd.to_datetime(
        AS["actual_dep_time"],
        format="%H:%M",
        errors="coerce"
    )
    .dt.time
)
# Convert delay in minutes to Timedelta
AS["dep_delay_wheels_off"] = (
    pd.to_datetime(
        AS["dep_delay_wheels_off"],
        format="%H:%M",
        errors="coerce"
    )
    .dt.time
)

AS['dep_delay_wheels_off_hour'] = AS['dep_delay_wheels_off'].apply(lambda x: x.hour if pd.notnull(x) else None)

# Convert the new 'dep_delay_wheels_off_hour' column to numeric type
AS['dep_delay_wheels_off_hour'] = pd.to_numeric(AS['dep_delay_wheels_off_hour'], errors='coerce')


numeric_cols = [
    'scheduled_elapsed', 'actual_elapsed',
    'dep_delay', 'taxi_out_time', 'delay_carrier',
    'delay_weather', 'delay_nas', 'delay_security', 'delay_late_aircraft'
]

for col in numeric_cols:
    AS[col] = pd.to_numeric(AS[col], errors='coerce')

# Handle missing data
AS.dropna(subset=['flight_date', 'flight_number', 'destination_airport'], inplace=True)

# Remove duplicates
AS.drop_duplicates(inplace=True)

# Final check
AS.info()
AS.head()
AS.to_csv('gdrive/My Drive/clean_data/AS_new.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 36877 entries, 0 to 36876
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Carrier Code               36877 non-null  object        
 1   flight_date                36877 non-null  datetime64[ns]
 2   flight_number              36877 non-null  float64       
 3   tail_number                36877 non-null  object        
 4   destination_airport        36877 non-null  object        
 5   scheduled_dep_time         36877 non-null  object        
 6   actual_dep_time            36877 non-null  object        
 7   scheduled_elapsed          36877 non-null  float64       
 8   actual_elapsed             36877 non-null  float64       
 9   dep_delay                  36877 non-null  float64       
 10  dep_delay_wheels_off       36875 non-null  object        
 11  taxi_out_time              36877 non-null  float64       
 12  delay_car

In [34]:
AS['month'] = AS['flight_date'].dt.month

# Extract the day of the month (1–31)
AS['day'] = AS['flight_date'].dt.day

AS.to_csv('gdrive/My Drive/clean_data_v2/AS.csv', index=False)

### DL

In [35]:
DL.rename(
    columns={
        "Carrier Code (IATA)": "carrier_code",
        "Date (MM/DD/YYYY)": "flight_date",
        "Flight Number": "flight_number",
        "Tail Number": "tail_number",
        "Destination Airport": "destination_airport",
        "Scheduled departure time": "scheduled_dep_time",
        "Actual departure time": "actual_dep_time",
        "Scheduled elapsed time (Minutes)": "scheduled_elapsed",
        "Actual elapsed time (Minutes)": "actual_elapsed",
        "Departure delay (Minutes)": "dep_delay",
        "Wheels-off time": "dep_delay_wheels_off",  # Updated key to match CSV header
        "Taxi-Out time (Minutes)": "taxi_out_time",
        "Delay Carrier (Minutes)": "delay_carrier",
        "Delay Weather (Minutes)": "delay_weather",
        "Delay National Aviation System (Minutes)": "delay_nas",
        "Delay Security (Minutes)": "delay_security",
        "Delay Late Aircraft Arrival (Minutes)": "delay_late_aircraft"
    },
    inplace=True
)

DL['flight_date'] = pd.to_datetime(DL['flight_date'], format='%m/%d/%Y', errors='coerce')
DL['flight_number'] = pd.to_numeric(DL['flight_number'], errors='coerce')



DL["scheduled_dep_time"] = DL["scheduled_dep_time"].astype(str).str.strip()
DL["scheduled_dep_time"] = (
    pd.to_datetime(DL["scheduled_dep_time"], format="%H:%M", errors="coerce")
    .dt.time
)

###################

DL['scheduled_dep_hour'] = DL['scheduled_dep_time'].apply(
    lambda t: t.hour if pd.notnull(t) else None
)

# (Optional) Convert the new column to numeric (int) if desired
DL['scheduled_dep_hour'] = pd.to_numeric(DL['scheduled_dep_hour'], errors='coerce')

# (Optional) If you want a true integer column that can handle missing values,
# convert to a nullable integer type:
DL['scheduled_dep_hour'] = DL['scheduled_dep_hour'].astype('Int64')



##################

DL["actual_dep_time"] = (
    pd.to_datetime(
        DL["actual_dep_time"],
        format="%H:%M",
        errors="coerce"
    )
    .dt.time
)

# Convert to numeric in case there are any stray strings
DL["dep_delay_wheels_off"] = pd.to_datetime(
    DL["dep_delay_wheels_off"], format="%H:%M", errors="coerce"
).dt.time

DL['dep_delay_wheels_off_hour'] = DL['dep_delay_wheels_off'].apply(lambda x: x.hour if pd.notnull(x) else None)

# Convert the new 'dep_delay_wheels_off_hour' column to numeric type
DL['dep_delay_wheels_off_hour'] = pd.to_numeric(DL['dep_delay_wheels_off_hour'], errors='coerce')


numeric_cols = [
    'scheduled_elapsed', 'actual_elapsed',
    'dep_delay', 'taxi_out_time', 'delay_carrier',
    'delay_weather', 'delay_nas', 'delay_security', 'delay_late_aircraft'
]

for col in numeric_cols:
    DL[col] = pd.to_numeric(DL[col], errors='coerce')

# Handle missing data
DL.dropna(subset=['flight_date', 'flight_number', 'destination_airport'], inplace=True)

# Remove duplicates
DL.drop_duplicates(inplace=True)

# Final check
DL.info()
DL.head()
DL.to_csv('gdrive/My Drive/clean_data/DL_new.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 132614 entries, 0 to 132613
Data columns (total 19 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Carrier Code               132614 non-null  object        
 1   flight_date                132614 non-null  datetime64[ns]
 2   flight_number              132614 non-null  float64       
 3   tail_number                132597 non-null  object        
 4   destination_airport        132614 non-null  object        
 5   scheduled_dep_time         132614 non-null  object        
 6   actual_dep_time            132529 non-null  object        
 7   scheduled_elapsed          132614 non-null  float64       
 8   actual_elapsed             132614 non-null  float64       
 9   dep_delay                  132614 non-null  float64       
 10  dep_delay_wheels_off       132566 non-null  object        
 11  taxi_out_time              132614 non-null  float64      

In [36]:
DL['month'] = DL['flight_date'].dt.month

# Extract the day of the month (1–31)
DL['day'] = DL['flight_date'].dt.day

DL.to_csv('gdrive/My Drive/clean_data_v2/DL.csv', index=False)

### WN

In [37]:
WN.rename(
    columns={
        "Carrier Code (IATA)": "carrier_code",
        "Date (MM/DD/YYYY)": "flight_date",
        "Flight Number": "flight_number",
        "Tail Number": "tail_number",
        "Destination Airport": "destination_airport",
        "Scheduled departure time": "scheduled_dep_time",
        "Actual departure time": "actual_dep_time",
        "Scheduled elapsed time (Minutes)": "scheduled_elapsed",
        "Actual elapsed time (Minutes)": "actual_elapsed",
        "Departure delay (Minutes)": "dep_delay",
        "Wheels-off time": "dep_delay_wheels_off",  # Updated key to match CSV header
        "Taxi-Out time (Minutes)": "taxi_out_time",
        "Delay Carrier (Minutes)": "delay_carrier",
        "Delay Weather (Minutes)": "delay_weather",
        "Delay National Aviation System (Minutes)": "delay_nas",
        "Delay Security (Minutes)": "delay_security",
        "Delay Late Aircraft Arrival (Minutes)": "delay_late_aircraft"
    },
    inplace=True
)

WN['flight_date'] = pd.to_datetime(WN['flight_date'], format='%m/%d/%Y', errors='coerce')
WN['flight_number'] = pd.to_numeric(WN['flight_number'], errors='coerce')



WN["scheduled_dep_time"] = WN["scheduled_dep_time"].astype(str).str.strip()
WN["scheduled_dep_time"] = (
    pd.to_datetime(WN["scheduled_dep_time"], format="%H:%M", errors="coerce")
    .dt.time
)

###################

WN['scheduled_dep_hour'] = WN['scheduled_dep_time'].apply(
    lambda t: t.hour if pd.notnull(t) else None
)

# (Optional) Convert the new column to numeric (int) if desired
WN['scheduled_dep_hour'] = pd.to_numeric(WN['scheduled_dep_hour'], errors='coerce')

# (Optional) If you want a true integer column that can handle missing values,
# convert to a nullable integer type:
WN['scheduled_dep_hour'] = WN['scheduled_dep_hour'].astype('Int64')



##################

WN["actual_dep_time"] = (
    pd.to_datetime(
        WN["actual_dep_time"],
        format="%H:%M",
        errors="coerce"
    )
    .dt.time
)

# Convert to numeric in case there are any stray strings
WN["dep_delay_wheels_off"] = (
    pd.to_datetime(
        WN["dep_delay_wheels_off"],  # e.g., "07:15"
        format="%H:%M",
        errors="coerce"
    )
    .dt.time  # yields a Python datetime.time object
)


WN['dep_delay_wheels_off_hour'] = WN['dep_delay_wheels_off'].apply(lambda x: x.hour if pd.notnull(x) else None)

# Convert the new 'dep_delay_wheels_off_hour' column to numeric type
WN['dep_delay_wheels_off_hour'] = pd.to_numeric(WN['dep_delay_wheels_off_hour'], errors='coerce')



numeric_cols = [
    'scheduled_elapsed', 'actual_elapsed',
    'dep_delay', 'taxi_out_time', 'delay_carrier',
    'delay_weather', 'delay_nas', 'delay_security', 'delay_late_aircraft'
]

for col in numeric_cols:
    WN[col] = pd.to_numeric(WN[col], errors='coerce')

# Handle missing data
WN.dropna(subset=['flight_date', 'flight_number', 'destination_airport'], inplace=True)

# Remove duplicates
WN.drop_duplicates(inplace=True)

# Final check
WN.info()
WN.head()
WN.to_csv('gdrive/My Drive/clean_data/WN_new.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 101619 entries, 0 to 101618
Data columns (total 19 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Carrier Code               101619 non-null  object        
 1   flight_date                101619 non-null  datetime64[ns]
 2   flight_number              101619 non-null  float64       
 3   tail_number                101449 non-null  object        
 4   destination_airport        101619 non-null  object        
 5   scheduled_dep_time         101619 non-null  object        
 6   actual_dep_time            101618 non-null  object        
 7   scheduled_elapsed          101619 non-null  float64       
 8   actual_elapsed             101619 non-null  float64       
 9   dep_delay                  101619 non-null  float64       
 10  dep_delay_wheels_off       101616 non-null  object        
 11  taxi_out_time              101619 non-null  float64      

In [38]:
WN['month'] = WN['flight_date'].dt.month

# Extract the day of the month (1–31)
WN['day'] = WN['flight_date'].dt.day

WN.to_csv('gdrive/My Drive/clean_data_v2/WN.csv', index=False)

### UA

In [39]:
UA.head()
UA = UA.loc[:, ~UA.columns.str.startswith("Unnamed")]

In [40]:
UA.rename(
    columns={
        "Carrier Code (IATA)": "carrier_code",
        "Date (MM/DD/YYYY)": "flight_date",
        "Flight Number": "flight_number",
        "Tail Number": "tail_number",
        "Destination Airport": "destination_airport",
        "Scheduled departure time": "scheduled_dep_time",
        "Actual departure time": "actual_dep_time",
        "Scheduled elapsed time (Minutes)": "scheduled_elapsed",
        "Actual elapsed time (Minutes)": "actual_elapsed",
        "Departure delay (Minutes)": "dep_delay",
        "Wheels-off time": "dep_delay_wheels_off",  # Updated key to match CSV header
        "Taxi-Out time (Minutes)": "taxi_out_time",
        "Delay Carrier (Minutes)": "delay_carrier",
        "Delay Weather (Minutes)": "delay_weather",
        "Delay National Aviation System (Minutes)": "delay_nas",
        "Delay Security (Minutes)": "delay_security",
        "Delay Late Aircraft Arrival (Minutes)": "delay_late_aircraft"
    },
    inplace=True
)

UA['flight_date'] = pd.to_datetime(UA['flight_date'], format='%m/%d/%Y', errors='coerce')
UA['flight_number'] = pd.to_numeric(UA['flight_number'], errors='coerce')



UA["scheduled_dep_time"] = UA["scheduled_dep_time"].astype(str).str.strip()
UA["scheduled_dep_time"] = (
    pd.to_datetime(UA["scheduled_dep_time"], format="%H:%M", errors="coerce")
    .dt.time
)

###################

UA['scheduled_dep_hour'] = UA['scheduled_dep_time'].apply(
    lambda t: t.hour if pd.notnull(t) else None
)

# (Optional) Convert the new column to numeric (int) if desired
UA['scheduled_dep_hour'] = pd.to_numeric(UA['scheduled_dep_hour'], errors='coerce')

# (Optional) If you want a true integer column that can handle missing values,
# convert to a nullable integer type:
UA['scheduled_dep_hour'] = UA['scheduled_dep_hour'].astype('Int64')



##################

UA["actual_dep_time"] = (
    pd.to_datetime(
        UA["actual_dep_time"],
        format="%H:%M",
        errors="coerce"
    )
    .dt.time
)

# Convert to numeric in case there are any stray strings
UA["dep_delay_wheels_off"] = (
    pd.to_datetime(
        UA["dep_delay_wheels_off"],  # e.g., "07:15"
        format="%H:%M",
        errors="coerce"
    )
    .dt.time  # yields a Python datetime.time object
)

UA['dep_delay_wheels_off_hour'] = UA['dep_delay_wheels_off'].apply(lambda x: x.hour if pd.notnull(x) else None)

# Convert the new 'dep_delay_wheels_off_hour' column to numeric type
UA['dep_delay_wheels_off_hour'] = pd.to_numeric(UA['dep_delay_wheels_off_hour'], errors='coerce')



numeric_cols = [
    'scheduled_elapsed', 'actual_elapsed',
    'dep_delay', 'taxi_out_time', 'delay_carrier',
    'delay_weather', 'delay_nas', 'delay_security', 'delay_late_aircraft'
]

for col in numeric_cols:
    UA[col] = pd.to_numeric(UA[col], errors='coerce')

# Handle missing data
UA.dropna(subset=['flight_date', 'flight_number', 'destination_airport'], inplace=True)

# Remove duplicates
UA.drop_duplicates(inplace=True)

# Final check
UA.info()
UA.head()
UA.to_csv('gdrive/My Drive/clean_data/UA_new.csv', index=False)

<ipython-input-40-22f59e2747d9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  UA.rename(
<ipython-input-40-22f59e2747d9>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  UA['flight_date'] = pd.to_datetime(UA['flight_date'], format='%m/%d/%Y', errors='coerce')
<ipython-input-40-22f59e2747d9>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95798 entries, 0 to 95797
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Carrier Code               95798 non-null  object        
 1   flight_date                95798 non-null  datetime64[ns]
 2   flight_number              95798 non-null  int64         
 3   tail_number                94590 non-null  object        
 4   destination_airport        95798 non-null  object        
 5   scheduled_dep_time         95798 non-null  object        
 6   actual_dep_time            95798 non-null  object        
 7   scheduled_elapsed          95798 non-null  int64         
 8   actual_elapsed             95798 non-null  int64         
 9   dep_delay                  95798 non-null  int64         
 10  dep_delay_wheels_off       95798 non-null  object        
 11  taxi_out_time              95798 non-null  int64         
 12  dela

In [41]:
UA['month'] = UA['flight_date'].dt.month

# Extract the day of the month (1–31)
UA['day'] = UA['flight_date'].dt.day

UA.to_csv('gdrive/My Drive/clean_data_v2/UA.csv', index=False)

<ipython-input-41-e1389964c799>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  UA['month'] = UA['flight_date'].dt.month
<ipython-input-41-e1389964c799>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  UA['day'] = UA['flight_date'].dt.day


SA


In [42]:
SA.head()
SA = SA.loc[:, ~SA.columns.str.startswith("Unnamed")]

In [43]:
SA.rename(
    columns={
        "Carrier Code (IATA)": "carrier_code",
        "Date (MM/DD/YYYY)": "flight_date",
        "Flight Number": "flight_number",
        "Tail Number": "tail_number",
        "Destination Airport": "destination_airport",
        "Scheduled departure time": "scheduled_dep_time",
        "Actual departure time": "actual_dep_time",
        "Scheduled elapsed time (Minutes)": "scheduled_elapsed",
        "Actual elapsed time (Minutes)": "actual_elapsed",
        "Departure delay (Minutes)": "dep_delay",
        "Wheels-off time": "dep_delay_wheels_off",  # Updated key to match CSV header
        "Taxi-Out time (Minutes)": "taxi_out_time",
        "Delay Carrier (Minutes)": "delay_carrier",
        "Delay Weather (Minutes)": "delay_weather",
        "Delay National Aviation System (Minutes)": "delay_nas",
        "Delay Security (Minutes)": "delay_security",
        "Delay Late Aircraft Arrival (Minutes)": "delay_late_aircraft"
    },
    inplace=True
)

SA['flight_date'] = pd.to_datetime(SA['flight_date'], format='%m/%d/%Y', errors='coerce')
SA['flight_number'] = pd.to_numeric(SA['flight_number'], errors='coerce')



SA["scheduled_dep_time"] = SA["scheduled_dep_time"].astype(str).str.strip()
SA["scheduled_dep_time"] = (
    pd.to_datetime(SA["scheduled_dep_time"], format="%H:%M", errors="coerce")
    .dt.time
)

###################

SA['scheduled_dep_hour'] = SA['scheduled_dep_time'].apply(
    lambda t: t.hour if pd.notnull(t) else None
)

# (Optional) Convert the new column to numeric (int) if desired
SA['scheduled_dep_hour'] = pd.to_numeric(SA['scheduled_dep_hour'], errors='coerce')

# (Optional) If you want a true integer column that can handle missing values,
# convert to a nullable integer type:
SA['scheduled_dep_hour'] = SA['scheduled_dep_hour'].astype('Int64')



##################

SA["actual_dep_time"] = (
    pd.to_datetime(
        SA["actual_dep_time"],
        format="%H:%M",
        errors="coerce"
    )
    .dt.time
)

# Convert to numeric in case there are any stray strings
SA["dep_delay_wheels_off"] = (
    pd.to_datetime(
        SA["dep_delay_wheels_off"],  # e.g., "07:15"
        format="%H:%M",
        errors="coerce"
    )
    .dt.time  # yields a Python datetime.time object
)

SA['dep_delay_wheels_off_hour'] = SA['dep_delay_wheels_off'].apply(lambda x: x.hour if pd.notnull(x) else None)

# Convert the new 'dep_delay_wheels_off_hour' column to numeric type
SA['dep_delay_wheels_off_hour'] = pd.to_numeric(SA['dep_delay_wheels_off_hour'], errors='coerce')



numeric_cols = [
    'scheduled_elapsed', 'actual_elapsed',
    'dep_delay', 'taxi_out_time', 'delay_carrier',
    'delay_weather', 'delay_nas', 'delay_security', 'delay_late_aircraft'
]

for col in numeric_cols:
    SA[col] = pd.to_numeric(SA[col], errors='coerce')

# Handle missing data
SA.dropna(subset=['flight_date', 'flight_number', 'destination_airport'], inplace=True)

# Remove duplicates
SA.drop_duplicates(inplace=True)

# Final check
SA.info()
SA.head()
SA.to_csv('gdrive/My Drive/clean_data/SA_new.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 43191 entries, 0 to 43190
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Carrier Code               43191 non-null  object        
 1   flight_date                43191 non-null  datetime64[ns]
 2   flight_number              43191 non-null  float64       
 3   tail_number                43191 non-null  object        
 4   destination_airport        43191 non-null  object        
 5   scheduled_dep_time         43191 non-null  object        
 6   actual_dep_time            43157 non-null  object        
 7   scheduled_elapsed          43191 non-null  float64       
 8   actual_elapsed             43191 non-null  float64       
 9   dep_delay                  43191 non-null  float64       
 10  dep_delay_wheels_off       43153 non-null  object        
 11  taxi_out_time              43191 non-null  float64       
 12  delay_car

In [44]:
SA['month'] = SA['flight_date'].dt.month

# Extract the day of the month (1–31)
SA['day'] = SA['flight_date'].dt.day

SA.to_csv('gdrive/My Drive/clean_data_v2/SA.csv', index=False)

JB

In [45]:
JB.head()
JB = JB.loc[:, ~JB.columns.str.startswith("Unnamed")]

In [46]:
JB.rename(
    columns={
        "Carrier Code (IATA)": "carrier_code",
        "Date (MM/DD/YYYY)": "flight_date",
        "Flight Number": "flight_number",
        "Tail Number": "tail_number",
        "Destination Airport": "destination_airport",
        "Scheduled departure time": "scheduled_dep_time",
        "Actual departure time": "actual_dep_time",
        "Scheduled elapsed time (Minutes)": "scheduled_elapsed",
        "Actual elapsed time (Minutes)": "actual_elapsed",
        "Departure delay (Minutes)": "dep_delay",
        "Wheels-off time": "dep_delay_wheels_off",  # Updated key to match CSV header
        "Taxi-Out time (Minutes)": "taxi_out_time",
        "Delay Carrier (Minutes)": "delay_carrier",
        "Delay Weather (Minutes)": "delay_weather",
        "Delay National Aviation System (Minutes)": "delay_nas",
        "Delay Security (Minutes)": "delay_security",
        "Delay Late Aircraft Arrival (Minutes)": "delay_late_aircraft"
    },
    inplace=True
)

JB['flight_date'] = pd.to_datetime(JB['flight_date'], format='%m/%d/%Y', errors='coerce')
JB['flight_number'] = pd.to_numeric(JB['flight_number'], errors='coerce')



JB["scheduled_dep_time"] = JB["scheduled_dep_time"].astype(str).str.strip()
JB["scheduled_dep_time"] = (
    pd.to_datetime(JB["scheduled_dep_time"], format="%H:%M", errors="coerce")
    .dt.time
)

###################

JB['scheduled_dep_hour'] = JB['scheduled_dep_time'].apply(
    lambda t: t.hour if pd.notnull(t) else None
)

# (Optional) Convert the new column to numeric (int) if desired
JB['scheduled_dep_hour'] = pd.to_numeric(JB['scheduled_dep_hour'], errors='coerce')

# (Optional) If you want a true integer column that can handle missing values,
# convert to a nullable integer type:
JB['scheduled_dep_hour'] = JB['scheduled_dep_hour'].astype('Int64')



##################

JB["actual_dep_time"] = (
    pd.to_datetime(
        JB["actual_dep_time"],
        format="%H:%M",
        errors="coerce"
    )
    .dt.time
)

# Convert to numeric in case there are any stray strings
JB["dep_delay_wheels_off"] = (
    pd.to_datetime(
        JB["dep_delay_wheels_off"],  # e.g., "07:15"
        format="%H:%M",
        errors="coerce"
    )
    .dt.time  # yields a Python datetime.time object
)

JB['dep_delay_wheels_off_hour'] = JB['dep_delay_wheels_off'].apply(lambda x: x.hour if pd.notnull(x) else None)

# Convert the new 'dep_delay_wheels_off_hour' column to numeric type
JB['dep_delay_wheels_off_hour'] = pd.to_numeric(JB['dep_delay_wheels_off_hour'], errors='coerce')



numeric_cols = [
    'scheduled_elapsed', 'actual_elapsed',
    'dep_delay', 'taxi_out_time', 'delay_carrier',
    'delay_weather', 'delay_nas', 'delay_security', 'delay_late_aircraft'
]

for col in numeric_cols:
    JB[col] = pd.to_numeric(JB[col], errors='coerce')

# Handle missing data
JB.dropna(subset=['flight_date', 'flight_number', 'destination_airport'], inplace=True)

# Remove duplicates
JB.drop_duplicates(inplace=True)

# Final check
JB.info()
JB.head()
JB.to_csv('gdrive/My Drive/clean_data/JB_new.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 41427 entries, 0 to 41426
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Carrier Code               41427 non-null  object        
 1   flight_date                41427 non-null  datetime64[ns]
 2   flight_number              41427 non-null  float64       
 3   tail_number                41427 non-null  object        
 4   destination_airport        41427 non-null  object        
 5   scheduled_dep_time         41427 non-null  object        
 6   actual_dep_time            41388 non-null  object        
 7   scheduled_elapsed          41427 non-null  float64       
 8   actual_elapsed             41427 non-null  float64       
 9   dep_delay                  41427 non-null  float64       
 10  dep_delay_wheels_off       41373 non-null  object        
 11  taxi_out_time              41427 non-null  float64       
 12  delay_car

In [47]:
SA['month'] = JB['flight_date'].dt.month

# Extract the day of the month (1–31)
JB['day'] = JB['flight_date'].dt.day

JB.to_csv('gdrive/My Drive/clean_data_v2/JB.csv', index=False)